In [1]:
import sys
sys.path.append('../utils')

In [2]:
import pipeline
import importlib
import pandas as pd

importlib.reload(pipeline)
from pipeline import extract_session_data, clean_session_data, get_driver_laps, get_manual_finish_dict
from pipeline import assemble_race_dataset, extract_fp2_features, extract_quali_features, assemble_race_dataset_pre_race
from fastf1 import get_session

In [3]:
# Italian GP 2025 (Round 16)
# This cell was made before the race so it excludes the race finish data 
df_fp2_16 = extract_session_data(2025, "Monza", "FP2")
df_quali_16 = extract_session_data(2025, "Monza", "Q")

fp2_clean_16 = clean_session_data(df_fp2_16)
quali_clean_16 = clean_session_data(df_quali_16)

fp2_features_16 = extract_fp2_features(fp2_clean_16)

quali_features_16 = extract_quali_features(quali_clean_16)
                       
race_df_16 = assemble_race_dataset_pre_race(fp2_features_16, quali_features_16)
race_df_16["grand_prix"] = "Monza"
race_df_16

req         WARNING 	DEFAULT CACHE ENABLED! (4.08 GB) /Users/illiapolishchuk/Library/Caches/fastf1
core           INFO 	Loading data for Italian Grand Prix - Practice 2 [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        

,Driver,fp2_avg_lap,fp2_best_lap,fp2_total_laps,FastestQualiLap,QualiPosition,grand_prix
0,ALB,97.198375,80.179,16,79.583,14,Monza
1,ALO,96.188471,80.645,17,79.362,9,Monza
2,ANT,114.051500,81.367,2,79.200,7,Monza
3,BEA,92.480750,80.607,20,79.446,11,Monza
4,BOR,86.880300,80.475,10,79.323,8,Monza
5,COL,88.594100,81.564,20,79.992,18,Monza
6,GAS,91.927409,81.102,22,80.103,19,Monza
7,HAD,87.386385,80.383,13,79.917,16,Monza
8,HAM,93.646687,80.070,16,79.124,5,Monza
9,HUL,88.831000,80.241,15,79.498,12,Monza


In [4]:
#race_df_16.to_csv("../data/r16_monza_2025.csv", index=False)

The .csv file was edited to account for Hammilton's grid penalty. although he qualified P5, he will be starting the race P10.

In [1]:
import joblib

model  = joblib.load("../models/logreg_scaled_quali_pre_r16_monza.pkl")
scaler = joblib.load("../models/scaler_pre_r16_monza.pkl")

In [2]:
import pandas as pd

df_netherlands = pd.read_csv('../data/r16_monza_2025.csv')
df_netherlands.head()

,Driver,fp2_avg_lap,fp2_best_lap,fp2_total_laps,FastestQualiLap,QualiPosition,grand_prix
0,ALB,97.198375,80.179,16,79.583,14,Monza
1,ALO,96.188471,80.645,17,79.362,8,Monza
2,ANT,114.051500,81.367,2,79.200,6,Monza
3,BEA,92.480750,80.607,20,79.446,11,Monza
4,BOR,86.880300,80.475,10,79.323,7,Monza


In [3]:
feature_cols = ['fp2_avg_lap', 'fp2_best_lap', 'fp2_total_laps', 'FastestQualiLap', 'QualiPosition'] 
X_netherlands = df_netherlands[feature_cols]

X_netherlands_scaled = scaler.transform(X_netherlands) 
proba_podium = model.predict_proba(X_netherlands_scaled)[:, 1]

results = df_netherlands.copy()
results["podium_probability"] = proba_podium

results_sorted = results.sort_values(by="podium_probability", ascending=False)

predicted_podium = results_sorted[["Driver", "podium_probability"]]

In [4]:
predicted_podium

,Driver,podium_probability
19,VER,0.927614
12,NOR,0.866091
14,PIA,0.848658
11,LEC,0.770775
15,RUS,0.673828
2,ANT,0.564449
1,ALO,0.381252
4,BOR,0.224781
18,TSU,0.218648
8,HAM,0.167828


In [5]:
predicted_podium.head(3)

,Driver,podium_probability
19,VER,0.927614
12,NOR,0.866091
14,PIA,0.848658
